In [ ]:
import numpy as np 
import pandas as pd 
import math
import datetime

from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

from sklearn.ensemble import GradientBoostingRegressor


from catboost import CatBoostRegressor, Pool
from sklearn.preprocessing import LabelEncoder

from numpy import mean, median
from sklearn.model_selection import GridSearchCV

from warnings import simplefilter
simplefilter("ignore")

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import HistGradientBoostingRegressor
from xgboost import XGBRegressor
from catboost import CatBoostRegressor, Pool
from lightgbm import LGBMRegressor

In [ ]:
df_train = pd.read_csv('../input/tabular-playground-series-mar-2022/train.csv', index_col="row_id", parse_dates=['time'])
df_test = pd.read_csv('../input/tabular-playground-series-mar-2022/test.csv', index_col="row_id", parse_dates=['time'])
df_train_idx = df_train.index
df_test_idx = df_test.index

In [ ]:
df_train['road'] = df_train['x'].astype(str) + df_train['y'].astype(str) + df_train['direction']
df_test['road']  = df_test['x'].astype(str) + df_test['y'].astype(str) + df_test['direction']

le = LabelEncoder()
df_train['road'] = le.fit_transform(df_train['road'])
df_test['road']  = le.transform(df_test['road'])

In [ ]:
minute = df_train.copy()
minute['time'] = minute['time'] + pd.Timedelta(20, unit="m")
minute = minute.rename(columns={'congestion':'lag'})[['time','direction','road', 'lag']]
df_train = df_train.merge(minute, on=['time','direction','road'], how='left')
df_test = df_test.merge(minute, on=['time','direction','road'], how='left')

In [ ]:
df_train.head(500)

In [ ]:
df_train['time'][848834] - df_train['time'][0]

In [ ]:
def add_datetime_features(df):
    df['month']   = df['time'].dt.month
    df['day']     = df['time'].dt.day
    df['weekday'] = df['time'].dt.weekday
    df['weekend'] = (df['time'].dt.weekday >= 5)
    df['hour']    = df['time'].dt.hour
    df['minute']  = df['time'].dt.minute
    df['afternoon'] = df['hour'] >= 12
    
    # combination of hour and minute features
    df['daytime_id'] = ( ( df.time.dt.hour*60 + df.time.dt.minute ) /20 ).astype(int)

In [ ]:
add_datetime_features(df_train)
add_datetime_features(df_test)

In [ ]:
median = df_train.groupby(['road', 'daytime_id']).congestion.median().astype(int)

In [ ]:
df=df_train.copy()
df = df_train.merge(median, left_on=['road', 'daytime_id'], right_index=True)

In [ ]:
df2 = df_test.copy()
df2 = df_test.merge(median,left_on=['road', 'daytime_id'], right_index=True)

In [ ]:
medt = df2['congestion']
df_test['median'] = medt

In [ ]:
df_test.head()

In [ ]:
med = df['congestion_y']
df_train['median'] = med
df_train['lag'] = df_train['lag'].fillna(df_train['median']) 
df_test['lag'] = df_test['lag'].fillna(df_test['median'])

In [ ]:
df_train.head()

In [ ]:
X_train = df_train.copy()
y_train = df_train['congestion']
X_train = X_train.drop(['congestion','x','y','direction','time'],axis=1)

In [ ]:
from sklearn.feature_selection import mutual_info_regression

mi_scores = mutual_info_regression(X_train, y_train)
mi_scores = pd.Series(mi_scores, name="MI_score", index=X_train.columns)
mi_scores = mi_scores.sort_values(ascending=False)
df_mi_scores = pd.DataFrame(mi_scores).reset_index().rename(columns={'index':'feature'})
df_mi_scores

The 20 minute lag and median seem to be the most informative features

Will use the median congestion and the congestion value 20 minutes prior to predict test congestions. See [notebook](https://www.kaggle.com/code/robertturro/tps-march-2022-useful-visuals) for EDA and visuals

In [ ]:
y_train = df_train['congestion']
train = df_train.drop(['time','direction','month','day','weekday','weekend','hour','minute','afternoon','road','daytime_id','congestion'],axis=1)
test = df_test.drop(['time','direction','month','day','weekday','weekend','hour','minute','afternoon','road','daytime_id'],axis=1)

In [ ]:
def get_params(df):
    mini = min(df['mean_test_score'])
    for row in range(len(df)):
        if df.iloc[row]['mean_test_score'] == mini:
            return df.iloc[row]['params']

In [ ]:
cat = GridSearchCV(CatBoostRegressor(logging_level='Silent', eval_metric='MAE', loss_function='MAE', random_state=42),{
    'n_estimators' : [50,100,1000],
    'learning_rate' : [1.0,0.1,0.01]
}, scoring = 'neg_mean_squared_error',return_train_score=False)

ada = GridSearchCV(AdaBoostRegressor(random_state=42),{
    'n_estimators' : [50,100,1000],
    'learning_rate' : [1.0,0.1,0.01]
}, scoring = 'neg_mean_squared_error',return_train_score=False)

cat.fit(train,y_train)
ada.fit(train,y_train)

cat_df = pd.DataFrame(cat.cv_results_)
ada_df = pd.DataFrame(ada.cv_results_)


print(f'Cat:{ max(cat_df["mean_test_score"]) }, Ada:{max(ada_df["mean_test_score"])}')

In [ ]:
p1 = get_params(cat_df)
p2 = get_params(ada_df)

In [ ]:
cb = CatBoostRegressor(p1)
ab = AdaBoostRegressor(p2)

cb.fit(train,y_train)
ab.fit(train,y_train)

In [ ]:
cat_pred = cb.predict(test)
ada_pred = ab.predict(test)

In [ ]:
sample_submission = pd.read_csv('../input/tabular-playground-series-mar-2022/sample_submission.csv', index_col="row_id")

In [ ]:
sample_submission['cat'] = cat_pred
sample_submission['ada'] = ada_pred

In [ ]:
sample_submission['congestion'] = sample_submission.mean(axis=1)

In [ ]:
sample_submission = sample_submission.astype(int)
sub = sample_submission.copy()
sub = sub.drop(['cat','ada'],axis=1)

In [ ]:
df_test = df_test.set_index(df_test_idx)
df_test['congestion'] = sample_submission['congestion']

From [TPSMAR22 Generalizing the Special Values](https://www.kaggle.com/code/ambrosm/tpsmar22-generalizing-the-special-values)

In [ ]:
submission_in = sub.copy()

sep = df_train[(df_train.time.dt.hour >= 12) & (df_train.time.dt.weekday < 5) &
            (df_train.time.dt.dayofyear >= 246)]
lower = sep.groupby(['hour', 'minute','x','y','direction']).congestion.quantile(0.15).values
upper = sep.groupby(['hour', 'minute','y','x','direction']).congestion.quantile(0.7).values

In [ ]:
submission_out = submission_in.copy()
submission_out['congestion'] = submission_in.congestion.clip(lower, upper)

# Submission

In [ ]:
submission_out['congestion'] = submission_out['congestion'].astype(int)

In [ ]:
submission_out = submission_out.reset_index()

In [ ]:
submission_out.to_csv('submission.csv',index=False)